In [ ]:
import tensorflow as tf
from tensorflow import keras

import numpy as np

print(tf.__version__)

In [ ]:
tf.constant([u"You're", u"welcome!"]).shape

In [ ]:
text_utf8 = tf.constant(u"语言处理")
text_utf8

In [ ]:
text_utf16be = tf.constant(u"语言处理".encode("UTF-16-BE"))
text_utf16be

In [ ]:
text_chars = tf.constant([ord(char) for char in u"语言处理"])
text_chars


In [ ]:
tf.strings.unicode_decode(text_utf8,
                          input_encoding='UTF-8')

In [ ]:
tf.strings.unicode_encode(text_chars,
                          output_encoding='UTF-8')

In [ ]:
tf.strings.unicode_transcode(text_utf8,
                             input_encoding='UTF8',
                             output_encoding='UTF-16-BE')

In [ ]:
batch_utf8 = [s.encode('UTF-8') for s in
              [u'hÃllo', u'What is the weather tomorrow', u'Göödnight', u'😊']]
batch_utf8

In [ ]:
batch_chars_ragged = tf.strings.unicode_decode(batch_utf8,
                                               input_encoding='UTF-8')

batch_chars_ragged                                             

In [ ]:
for sentence_chars in batch_chars_ragged.to_list():
  print(sentence_chars)
  

In [ ]:
batch_chars_padded = batch_chars_ragged.to_tensor(default_value=-1)
print(batch_chars_padded.numpy())

In [ ]:
batch_chars_sparse = batch_chars_ragged.to_sparse()
nrows, ncols = batch_chars_sparse.dense_shape.numpy()
batch_chars_sparse.shape
# ncols


In [ ]:
batch_chars_ragged.shape

In [ ]:
batch_chars_sparse = batch_chars_ragged.to_sparse()
batch_chars_sparse.shape

In [ ]:
batch_chars_sparse = batch_chars_ragged.to_sparse()

nrows, ncols = batch_chars_sparse.dense_shape.numpy()
elements = [['_' for i in range(ncols)] for j in range(nrows)]
for (row, col), value in zip(batch_chars_sparse.indices.numpy(), batch_chars_sparse.values.numpy()):
  elements[row][col] = str(value)
# max_width = max(len(value) for row in elements for value in row)
value_lengths = []
for row in elements:
  for value in row:
    value_lengths.append(len(value))
max_width = max(value_lengths)
print('[%s]' % '\n '.join(
    '[%s]' % ', '.join(value.rjust(max_width) for value in row)
    for row in elements))

In [ ]:
tf.strings.unicode_encode(
    tf.RaggedTensor.from_sparse(batch_chars_sparse),
    output_encoding='UTF-8')

In [ ]:
tf.strings.unicode_encode(
    tf.RaggedTensor.from_tensor(batch_chars_padded, padding=-1),
    output_encoding='UTF-8')

In [ ]:
# Note that the final character takes up 4 bytes in UTF8.
thanks = u'Thanks 😊'.encode('UTF-8')
num_bytes = tf.strings.length(thanks).numpy()
num_chars = tf.strings.length(thanks, unit='UTF8_CHAR').numpy()
print('{} bytes; {} UTF-8 characters'.format(num_bytes, num_chars))

In [ ]:
# Here, unit='BYTE' (default). Returns a single byte with len=1
tf.strings.substr(thanks, pos=7, len=1).numpy()

In [ ]:
print(tf.strings.substr(thanks, pos=7, len=1, unit='UTF8_CHAR').numpy())


In [ ]:
tf.strings.unicode_split(thanks, 'UTF-8').numpy()

In [ ]:
codepoints, offsets = tf.strings.unicode_decode_with_offsets(u'🎈🎉🎊', 'UTF-8')

for (codepoint, offset) in zip(codepoints.numpy(), offsets.numpy()):
  print('At byte offset {}: codepoint {}'.format(offset, codepoint))

In [ ]:
sentence_texts = [u'Hello, world.', u'世界こんにちは']

In [ ]:
sentence_char_codepoint = tf.strings.unicode_decode(sentence_texts, 'UTF-8')
print(sentence_char_codepoint)

In [ ]:
sentence_char_script = tf.strings.unicode_script(sentence_char_codepoint)
print(sentence_char_script)

In [ ]:
sentence_char_starts_word = tf.concat(
    [tf.fill([sentence_char_script.nrows(), 1], True),
     tf.not_equal(sentence_char_script[:, 1:], sentence_char_script[:, :-1])],
    axis=1)

# dtype: int64; shape: [num_words]
#
# word_starts[i] is the index of the character that starts the i'th word (in
# the flattened list of characters from all sentences).
word_starts = tf.squeeze(tf.where(sentence_char_starts_word.values), axis=1)
print(word_starts)

In [ ]:
word_char_codepoint = tf.RaggedTensor.from_row_starts(
    values=sentence_char_codepoint.values,
    row_starts=word_starts)
print(word_char_codepoint)

In [ ]:
sentence_num_words = tf.reduce_sum(
    tf.cast(sentence_char_starts_word, tf.int64),
    axis=1)

# dtype: int32; shape: [num_sentences, (num_words_per_sentence), (num_chars_per_word)]
#
# sentence_word_char_codepoint[i, j, k] is the codepoint for the k'th character
# in the j'th word in the i'th sentence.
sentence_word_char_codepoint = tf.RaggedTensor.from_row_lengths(
    values=word_char_codepoint,
    row_lengths=sentence_num_words)
print(sentence_word_char_codepoint)

tf.strings.unicode_encode(sentence_word_char_codepoint, 'UTF-8').to_list()